This file needs requires external data!

(Run on workstation)

In [8]:
import xarray as xr
import glob
import numpy as np

import sys

sys.path.append('../src')
import utils as ut
np.set_printoptions(precision=2)

In [9]:
run = 'hal2'

In [10]:
fnames = sorted(glob.glob(f'../data/ecefiles/{run}/{run}*T.nc'))

nyears = 0
for fname in fnames:
    ds = xr.open_dataset(fname)
    nyears += int(np.floor(len(ds.time_counter)/12))
    ds.close()
print(nyears, 'years')

17 years


In [11]:
#Get time-independent variables
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf'].values;
ds.close()

ds = xr.open_dataset(fnames[0])
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
levmid = ds['olevel'].values
lev = ds['olevel_bounds'].values
time_bnds = ds['time_centered_bounds']
thick = ds['e3t'].values #Quasi-time-independent, treated as fixed
ds.close()
secs = (time_bnds[:12,1]-time_bnds[:12,0]).values / np.timedelta64(1, 's')

In [12]:
#Get weighted mask for basin averages
lons = np.repeat(lon[np.newaxis,:,:],len(levmid),axis=0)
lats = np.repeat(lat[np.newaxis,:,:],len(levmid),axis=0)
mask = np.repeat(np.zeros(lons.shape)[np.newaxis,:,:,:],len(ut.basin),axis=0)
aweight = np.zeros(mask.shape)

for b,bas in enumerate(ut.basin):
    mm = np.zeros(lons.shape)
    if bas=='East Ant.':
        #EAIS
        mm[((lons<173) & (lons>-10)) & (lats<-65) & (lats>-76)] = 1
        depp = 369
    if bas=='Ross':
        #ROSS
        mm[((lons>150) | (lons<-150)) & (lats<-76)] = 1
        depp = 312        
    if bas=='Amundsen':
        #AMUN
        mm[(lons>-150) & (lons<-80) & (lats<-70)] = 1
        depp = 305
    if bas=='Weddell':
        #WEDD
        mm[(lons>-65) & (lons<-10) & (lats<-72)] = 1
        depp = 420
    if bas=='Peninsula':
        #APEN
        mm[(lons>-66) & (lons<-56) & (lats>-70) & (lats<-65)] = 1
        mm[(lons>-80) & (lons<-65) & (lats>-75) & (lats<-70)] = 1
        depp = 420
        
    z0 = depp-50.
    i0 = np.argmax(lev[:,1]>z0)
    mm[:i0,:,:] = 0
    w0 = (lev[i0,1]-z0)/(lev[i0,1]-lev[i0,0])
    mm[i0,:,:] = w0*mm[i0,:,:]
    for j in range(0,lon.shape[0]):
        for i in range(0,lon.shape[1]):
            if np.nansum(thick[0,i0:,j,i]) == 0:
                continue
            z1 = depp+50.
            i1 = np.argmin(lev[:,1]<z1)
            w1 = (z1-lev[i1,0])/(lev[i1,1]-lev[i1,0])
            mm[i1,j,i] = w1*mm[i1,j,i]
            mm[i1+1:,j,i] = 0
    mask[b,:,:,:] = mm*np.where(np.isnan(thick[0,:,:,:]),0,thick[0,:,:,:])/100.
    aweight[b,:,:,:] = mask[b,:,:,:]*area[np.newaxis,:,:] 

In [13]:
#Calculate basin-average annual time series
tbas = np.zeros((nyears,len(ut.basin)))
ttime = np.arange(nyears)
months = np.arange(0,12)

#Reference values from n012 for first year
print('ref: 0 [ 0.47  0.01  1.63 -1.28 -0.73]')
print('--------------------------------------')

c = 0
for f,fname in enumerate(fnames):
    ds = xr.open_dataset(fname)
    time = ds['time_centered'].values
    temp = ds['thetao'].values
    ds.close()
    
    year0 = int(fname[-27:-23])
    
    tb = np.zeros((len(ut.basin)))
    ny = int(len(time)/12)
    for y in np.arange(0,ny):
        for b,bas in enumerate(ut.basin):
            for m,mm in enumerate(months):
                tbb = np.nansum(temp[m+12*y,:,:,:]*aweight[b,:,:,:])/np.nansum(aweight[b,:,:,:])
                tb[b] += tbb*secs[m]
            tbas[c,b] = tb[b]/sum(secs)
        print(year0+y,c,tbas[c,:])
        tb = np.zeros((len(ut.basin)))
        c += 1

ref: 0 [ 0.47  0.01  1.63 -1.28 -0.73]
--------------------------------------
1950 0 [ 0.49  0.14  1.69 -1.31 -0.67]
1951 1 [ 0.44  0.41  1.72 -1.38 -0.72]
1952 2 [ 0.44  0.55  1.84 -1.42 -0.58]
1953 3 [ 0.48  0.83  1.98 -1.46 -0.56]
1954 4 [ 0.52  1.05  2.01 -1.48 -0.56]
1955 5 [ 0.6   1.04  1.91 -1.55 -0.67]
1956 6 [ 0.64  0.89  1.84 -1.52 -0.66]
1957 7 [ 0.66  0.81  1.9  -1.43 -0.65]
1958 8 [ 0.65  0.74  1.94 -1.35 -0.7 ]
1959 9 [ 0.67  0.98  1.84 -1.33 -0.82]
1960 10 [ 0.74  0.68  1.68 -1.24 -0.76]
1961 11 [ 0.72  0.41  1.68 -1.22 -0.66]
1962 12 [ 0.7   0.37  1.85 -1.24 -0.61]
1963 13 [ 0.68  0.67  1.95 -1.06 -0.49]
1964 14 [ 0.63  0.91  1.97 -1.04 -0.55]
1965 15 [ 0.65  0.96  1.91 -1.03 -0.45]
1966 16 [ 0.7   0.79  1.94 -1.05 -0.45]


In [14]:
temp2 = xr.DataArray(tbas,dims=('time','basin'),coords={'time':ttime,'basin':ut.basin},attrs={'unit':'degrees Celcius','long_name':'temperature time series per basin'})

ds = xr.Dataset({'temp':temp2})
ds.to_netcdf(f'../data/temperature_{run}.nc',mode='w')
ds.close()

In [8]:
mask2 = xr.DataArray(mask,dims=('basin','olevel','y','x'),coords={'basin':ut.basin,'olevel':levmid},attrs={'long_name':'basin mask'})
lat2  = xr.DataArray(lat,dims=('y','x'))
lon2  = xr.DataArray(lon,dims=('y','x'))

ds = xr.Dataset({'mask':mask2,'lat':lat2,'lon':lon2})
ds.to_netcdf(f'../data/basinmask.nc',mode='w')
ds.close()